In [1]:
import os
import sys
import time
import json
from datetime import datetime

import numpy as np
import pandas as pd

from models import models
from utils import read_df, read_numpy, write_df
import logging

from evaluate import rps

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
#Model imports
import numpy as np
import json
import os
import pandas as pd

from tqdm.auto import tqdm
from multiprocessing import Pool
from iteround import saferound
import scipy.stats as st
import tweedie
from category_encoders import OrdinalEncoder
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=50)

from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

from model import Model
from utils import read_df

INFO: Pandarallel will run on 50 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
DATASET_PATH = './dataset/processed/train_v1'
TEST_DATA_FILENAME = 'test_data.csv'
GROUND_TRUTH_FILENAME = 'test_ground_truth.npy'
TRAIN_DATA_FILENAME = 'train_data.parquet'
TRAIN_DATA_PROCESSED_FILENAME = 'train_data_features.parquet'
ITEM_DATA_FILEPATH = './dataset/items_static_metadata_full.jl'

TRAIN_DATA_X_FILENAME = 'train_data_x.parquet'
TRAIN_DATA_Y_FILENAME = 'train_data_y.parquet'
TRAIN_DATA_X_PROCESSED_FILENAME = 'train_data_x_features.parquet'
TRAIN_DATA_Y_PROCESSED_FILENAME = 'train_data_y_features.parquet'

TEST_FROMTRAIN_DATA_LAST29_FILENAME = 'test_fromtrain_data_last29.parquet'
TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME = 'test_fromtrain_data_last29_features.parquet'

In [4]:
dateset_indexes = [0]
model_name = 'simple_first_30_days_fixed_spike'

In [5]:
dataset_index = dateset_indexes[0]
dataset_current_path = os.path.join(DATASET_PATH, str(dataset_index))

test_data_filepath = os.path.join(dataset_current_path, TEST_DATA_FILENAME)
ground_truth_filepath = os.path.join(dataset_current_path, GROUND_TRUTH_FILENAME)
train_data_filepath = os.path.join(dataset_current_path, TRAIN_DATA_FILENAME)
train_data_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_PROCESSED_FILENAME)

train_data_x_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_FILENAME)
train_data_y_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_FILENAME)
train_data_x_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_X_PROCESSED_FILENAME)
train_data_y_processed_filepath = os.path.join(dataset_current_path, TRAIN_DATA_Y_PROCESSED_FILENAME)

test_fromtrain_data_last29_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_FILENAME)
test_fromtrain_data_last29_processed_filepath = os.path.join(dataset_current_path, TEST_FROMTRAIN_DATA_LAST29_PROCESSED_FILENAME)

In [6]:
#model = models['xgboost_v1']('./dataset')
#model.prepare_data()
#model.train()
#model.predict('./dataset/test_data.csv')

In [6]:
df_train = read_df(train_data_filepath)
df_item = read_df(ITEM_DATA_FILEPATH)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-1d52857e075c>", line 2, in <module>
    df_item = read_df(ITEM_DATA_FILEPATH)
  File "/home/edugarcia/kaggle/ml2021/utils.py", line 22, in read_df
    return pd.read_json(data, lines=True)
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/pandas/util/_decorators.py", line 199, in wrapper
    return func(*args, **kwargs)
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/pandas/util/_decorators.py", line 296, in wrapper
    return func(*args, **kwargs)
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/pandas/io/json/_json.py", line 618, in read_json
    result = json_reader.read()
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/pandas/io/json/_json.py", line 753, in read
    obj = self._get_object_parser(self._combine_lines(data

TypeError: object of type 'NoneType' has no len()

In [ ]:
df_test = read_df(test_data_filepath)
ground_truth = read_numpy(ground_truth_filepath)

df_train_processed = read_df(train_data_processed_filepath)

In [ ]:
#df_train_x = read_df(train_data_x_filepath)
#df_train_x = read_df(train_data_y_filepath)

df_test_fromtrain_x_features = read_df(test_fromtrain_data_last29_processed_filepath)
df_train_x_features = read_df(train_data_x_processed_filepath)
df_train_y_features = read_df(train_data_y_processed_filepath)

In [ ]:
x_df = df_train_x_features.sample(frac=1, random_state=42).reset_index(drop=True).copy()

In [ ]:
y_df = df_train_y_features.set_index('sku').loc[x_df['sku']].reset_index().copy()

In [11]:
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

In [12]:
id_column = 'sku'
sort_column = 'date'
numeric_columns = ['sold_quantity', 'current_price', 'minutes_active']

features = [id_column] + [sort_column] + numeric_columns

In [17]:
df = df_train[features]
df = df.sort_values(['sku', 'date']).reset_index(drop=True)
df_first_entry = df[df['sku'].diff() != 0]

sku_split = []
for i in range(len(df_first_entry)-1):
    start_index = df_first_entry.index[i]
    end_index = df_first_entry.index[i+1]
    sku_split.append(df.iloc[start_index:end_index])
sku_split.append(df.iloc[end_index:])

In [9]:
#df = df_train[df_train['sku'] == df_train.iloc[0]['sku']]
df_train = df_train['features']

In [ ]:
import multiprocessing
# We must import this explicitly, it is not imported by the top-level
# multiprocessing module.
import multiprocessing.pool
import time

from random import randint


class NoDaemonProcess(multiprocessing.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

# We sub-class multiprocessing.pool.Pool instead of multiprocessing.Pool
# because the latter is only a wrapper function, not a proper class.
class MyPool(multiprocessing.pool.Pool):
    Process = NoDaemonProcess

In [ ]:
import time
from multiprocessing.dummy import Pool as ThreadPool
start_time = time.time()

extraction_settings = ComprehensiveFCParameters()

def create_features_per_sku(df_sku):
    x_features_extracted = extract_features(df_sku, column_id='sku', column_sort='date',
                         default_fc_parameters=extraction_settings, n_jobs=0, disable_progressbar=True)
    return x_features_extracted

sku_data = []
with Pool(96) as p:
    for data in tqdm(p.imap(create_features_per_sku, sku_split), total=len(sku_split)):
        sku_data.append(data)

df_sku = pd.DataFrame(sku_data)
print("--- %s seconds ---" % (time.time() - start_time))

In [55]:
x_features_extracted

,minutes_active__variance_larger_than_standard_deviation,minutes_active__has_duplicate_max,minutes_active__has_duplicate_min,minutes_active__has_duplicate,minutes_active__sum_values,minutes_active__abs_energy,minutes_active__mean_abs_change,minutes_active__mean_change,minutes_active__mean_second_derivative_central,minutes_active__median,...,current_price__permutation_entropy__dimension_5__tau_1,current_price__permutation_entropy__dimension_6__tau_1,current_price__permutation_entropy__dimension_7__tau_1,current_price__query_similarity_count__query_None__threshold_0.0,"current_price__matrix_profile__feature_""min""__threshold_0.98","current_price__matrix_profile__feature_""max""__threshold_0.98","current_price__matrix_profile__feature_""mean""__threshold_0.98","current_price__matrix_profile__feature_""median""__threshold_0.98","current_price__matrix_profile__feature_""25""__threshold_0.98","current_price__matrix_profile__feature_""75""__threshold_0.98"
464801,0.0,1.0,1.0,1.0,84960.0,122342400.0,0.0,0.0,0.0,1440.0,...,0.965937,1.214718,1.431578,0.0,3.102753,6.447728,4.95205,5.078272,4.35793,5.504583


In [ ]:
x_feature_selection = select_features(x_df[numeric_columns], y_df['sold_quantity_mean'])
set(numeric_columns) - set(x_feature_selection.columns)

In [13]:
all_columns = ['item_domain_id', 'item_id', 'item_title', 'site_id', 'sku',
       'product_id', 'product_family_id', 'count', 'date_first', 'date_last',
       'date_diff', 'date_first_day', 'date_first_month', 'date_last_day',
       'date_last_month', 'sold_quantity_first', 'sold_quantity_last',
       'sold_quantity_sum', 'sold_quantity_mean', 'sold_quantity_std',
       'sold_quantity_min', 'sold_quantity_max', 'sold_quantity_mode',
       'sold_quantity_mode_tx', 'current_price_first', 'current_price_last',
       'current_price_sum', 'current_price_mean', 'current_price_std',
       'current_price_min', 'current_price_max', 'current_price_mode',
       'current_price_mode_tx', 'minutes_active_first', 'minutes_active_last',
       'minutes_active_sum', 'minutes_active_mean', 'minutes_active_std',
       'minutes_active_min', 'minutes_active_max', 'minutes_active_mode',
       'minutes_active_mode_tx', 'currency_first', 'currency_last',
       'currency_mode', 'currency_mode_tx', 'listing_type_first',
       'listing_type_last', 'listing_type_mode', 'listing_type_mode_tx',
       'shipping_logistic_type_first', 'shipping_logistic_type_last',
       'shipping_logistic_type_mode', 'shipping_logistic_type_mode_tx',
       'shipping_payment_first', 'shipping_payment_last',
       'shipping_payment_mode', 'shipping_payment_mode_tx',
       'minutes_active_series', 'current_price_series',
       'sold_quantity_series']

In [40]:
categorical_columns = ['site_id', 'item_id']
categorical_columns_with_nan = ['item_domain_id', 'product_id', 'product_family_id']
categorical_columns_composed = ['currency', 'listing_type', 'shipping_logistic_type', 'shipping_payment']
categorical_composed_suffix = ['_first', '_last', '_mode']

categorical_columns = categorical_columns + categorical_columns_with_nan
for category_name in categorical_columns_composed:
    for suffix in categorical_composed_suffix:
        categorical_columns.append(category_name+suffix)

numeric_columns = ['count', 'date_first_day', 'date_first_month', 'date_last_day', 'date_last_month']  
numeric_columns_composed = ['sold_quantity', 'current_price', 'minutes_active']
numeric_composed_suffix = ['_first', '_last', '_mode', '_mode_tx', '_sum', '_mean', '_std', '_min', '_max']

for numeric_name in numeric_columns_composed:
    for suffix in numeric_composed_suffix:
        numeric_columns.append(numeric_name+suffix)

categorical_to_numeric_composed_suffix = ['_mode_tx']
        
for category_name in categorical_columns_composed:
    for suffix in categorical_to_numeric_composed_suffix:
        numeric_columns.append(category_name+suffix)
        
series_columns = ['sold_quantity_series', 'current_price_series', 'minutes_active_series']
series_columns_headers = {}

series_columns_max = x_df['count'].max()
for series_name in series_columns:
    series_columns_headers[series_name] = [series_name+'_'+str(i) for i in range(series_columns_max)]
    numeric_columns = numeric_columns + series_columns_headers[series_name]

features = numeric_columns + categorical_columns

print('Number of features:', len(features), 'Non-used features: ', set(all_columns) - set(features) - set(series_columns))

Number of features: 140 Non-used features:  {'sku', 'date_first', 'date_last', 'item_title', 'date_diff'}


In [15]:
for column in numeric_columns_composed:
    x_df[column+'_std'] = x_df[column+'_std'].fillna(0)
    y_df[column+'_std'] = y_df[column+'_std'].fillna(0)

In [16]:
for column in categorical_columns_with_nan:    
    x_df[column] = x_df[column].astype(str).astype("category")
    y_df[column] = y_df[column].astype(str).astype("category")

In [17]:
categorical_pandas_order = {}

for column in categorical_columns:    
    x_df[column] = x_df[column].astype("category")
    categorical_pandas_order[column] = x_df[column].cat.categories
    y_df[column] = y_df[column].astype("category")
    y_df[column] = y_df[column].cat.set_categories(categorical_pandas_order[column], ordered=True)

In [18]:
for column in categorical_columns:
    x_df[column] = x_df[column].cat.codes
    y_df[column] = y_df[column].cat.codes

In [19]:
for column in  series_columns:
    def series_to_df(data):
        result = np.ones(series_columns_max)*-1
        data = json.loads(data)
        index_limit = len(data) if len(data) <= series_columns_max else series_columns_max
        result[-index_limit:] = data[-index_limit:]
        #return result
        return pd.Series(result, index=series_columns_headers[column])

    series_x_df = x_df[column].parallel_apply(series_to_df)
    series_y_df = y_df[column].parallel_apply(series_to_df)

    x_df = x_df.join(series_x_df)
    y_df = y_df.join(series_y_df)

In [20]:
X = x_df[features].values

In [21]:
#y = y_df[['sold_quantity_mean', 'sold_quantity_std']].values

In [22]:
y = np.array(list(y_df['sold_quantity_series'].apply(json.loads).values))
y = y.cumsum(axis=1)

In [34]:
SELECT_N = 1

index_list = []
y_ts = []
y_td = []

for i, t in enumerate(y):
    target_stocks, target_dates = np.unique(t, return_index=True)
    if target_stocks[0] == 0:
        target_stocks = target_stocks[1:]
        target_dates = target_dates[1:]
    size = len(target_stocks)
    index = np.arange(size)
    if size > SELECT_N:
        np.random.shuffle(index)
        index = index[:SELECT_N]
    
    for target_stock, target_date in zip(target_stocks[index], target_dates[index]):
        index_list.append(i)
        y_ts.append(target_stock)
        y_td.append(target_date)
        
index_list = np.array(index_list)
y_ts = np.array(y_ts)
y_td = np.array(y_td)

In [24]:
len(index_list)

1237446

In [25]:
X = X[index_list]
X = np.concatenate((X, np.reshape(y_ts, (-1, 1))), axis=1)

In [26]:
y = y_td

In [27]:
np.unique(y, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([50442, 48117, 45274, 41893, 39224, 37287, 41841, 43899, 43436,
        41162, 38633, 34867, 34733, 40971, 42642, 41596, 40853, 37383,
        34507, 34600, 41461, 43038, 43143, 42910, 40104, 36753, 36775,
        45297, 47599, 47006]))

{'count',
 'currency_mode_tx',
 'date_first_day',
 'date_last_day',
 'date_last_month',
 'shipping_payment_mode_tx'}

In [29]:
test_x_df = df_test_fromtrain_x_features.set_index('sku').loc[df_test['sku']].reset_index().copy()
for column in numeric_columns_composed:
    test_x_df[column+'_std'] = test_x_df[column+'_std'].fillna(0)
for column in categorical_columns_with_nan:    
    test_x_df[column] = test_x_df[column].astype(str).astype("category")
for column in categorical_columns:
    test_x_df[column] = test_x_df[column].astype("category")
    categorical_order = list(categorical_pandas_order[column])
    if len(test_x_df[column].cat.categories) != len(categorical_pandas_order[column]):
        for cat in set(test_x_df[column].cat.categories) - set(categorical_order):
                categorical_order.append(cat)
    test_x_df[column] = test_x_df[column].cat.set_categories(categorical_order, ordered=True)
    test_x_df[column] = test_x_df[column].cat.codes
for column in  series_columns:
    def series_to_df(data):
        result = np.ones(series_columns_max)*-1
        data = json.loads(data)
        index_limit = len(data) if len(data) <= series_columns_max else series_columns_max
        result[-index_limit:] = data[-index_limit:]
        #return result
        return pd.Series(result, index=series_columns_headers[column])

    series_x_test_df = test_x_df[column].parallel_apply(series_to_df)

    test_x_df = test_x_df.join(series_x_test_df)

X_test = test_x_df[features].values

In [30]:
X_test = np.concatenate((X_test, np.reshape(df_test['target_stock'].values, (-1, 1))), axis=1)

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from xgboost import XGBClassifier
#model = MultiOutputRegressor(XGBRegressor(
#                         n_estimators=800, max_depth=2, learning_rate=0.01,
#                         random_state=0, tree_method='gpu_hist',
#                         gpu_id=3), n_jobs=14)
model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, reg_lambda=5e-1,
                    random_state=0, tree_method='gpu_hist', objective='multi:softprob', num_class=30,
                    gpu_id=3)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)
y_pred = (y_pred/y_pred.sum(axis=1)[:,None]).round(4)
rps(y_pred, np.eye(30)[y_val.astype(int)])

/home/edugarcia/dependencies/xgboost/python-package/xgboost/sklearn.py:1204: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:01:06] WARNING: /home/edugarcia/dependencies/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-8ddfd8425afc>", line 9, in <module>
    model.fit(X_train, y_train)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/core.py", line 498, in inner_f
    return f(**kwargs)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/sklearn.py", line 1241, in fit
    callbacks=callbacks,
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/training.py", line 196, in train
    early_stopping_rounds=early_stopping_rounds)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/training.py", line 81, in _train_internal
    bst.update(dtrain, i, obj)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/core.py", line 1669, in update
    dtrain.handle))
KeyboardInterrupt

During handling of the abo

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/torch/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-32-8ddfd8425afc>", line 9, in <module>
    model.fit(X_train, y_train)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/core.py", line 498, in inner_f
    return f(**kwargs)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/sklearn.py", line 1241, in fit
    callbacks=callbacks,
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/training.py", line 196, in train
    early_stopping_rounds=early_stopping_rounds)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/training.py", line 81, in _train_internal
    bst.update(dtrain, i, obj)
  File "/home/edugarcia/dependencies/xgboost/python-package/xgboost/core.py", line 1669, in update
    dtrain.handle))
KeyboardInterrupt

During handling of the abo

n_estimators=1000, max_depth=2, learning_rate=0.08, 38.77598546656583
model = XGBRegressor(n_estimators=1000, max_depth=2, learning_rate=0.2, 38.27539167059256
n_estimators=1000, max_depth=4, learning_rate=0.1, 37.546678641103945
n_estimators=1000, max_depth=6, learning_rate=0.1, 37.40878445231095

In [32]:
from skopt import gp_minimize

def tune(params):
    print('params', params)
    model = XGBClassifier(n_estimators=params[0], learning_rate=params[1],
                       max_depth=params[2],
                       subsample=params[3],
                       colsample_bytree=params[4],
                       min_child_weight=params[5],
                       random_state=42, objective='multi:softprob', num_class=30,
                       tree_method='gpu_hist', gpu_id=3)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)
    y_pred = (y_pred/y_pred.sum(axis=1)[:,None]).round(4)
    return rps(y_pred, np.eye(30)[y_val.astype(int)])

space = [(100, 300),
         (1e-3, 5e-1, 'log-uniform'),
         (1, 10),
         (0.05, 0.95),
         (0.05, 0.95),
         (1,300)]
res = gp_minimize(tune, space, random_state=42, verbose=1, acq_optimizer="lbfgs", n_jobs=10)

ERROR! Session/line number was not unique in database. History logging moved to new session 170
Iteration No: 1 started. Evaluating function at random point.
params [259, 0.0031266912307463765, 8, 0.5871651421518383, 0.4512494775682321, 31]


KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(figsize=(20, 10))
#features_importances = model.estimators_[-1].feature_importances_
features_importances = model.feature_importances_
sort = np.flip(np.argsort(features_importances))[0:30]
ax.barh(np.array(features + ['target_stock'])[sort], features_importances[sort])
plt.show()

In [ ]:
variables_to_train = np.flip(np.argsort(features_importances))[0:15]
model = XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.2,
                    random_state=0, tree_method='gpu_hist', objective='multi:softprob', num_class=30,
                    gpu_id=3)
model.fit(X, y)
preds = model.predict_proba(X_test)
preds = (preds/preds.sum(axis=1)[:,None]).round(4)
rps(preds, ground_truth)

In [ ]:
model = XGBClassifier(n_estimators=1000, max_depth=6, learning_rate=0.1,
                    random_state=0, tree_method='gpu_hist', objective='multi:softprob', num_class=30,
                    gpu_id=3)
model.fit(X, y)
preds = model.predict_proba(X_test)
preds = (preds/preds.sum(axis=1)[:,None]).round(4)
rps(preds, ground_truth)

In [ ]:
def shift(arr, num, fill_value=0):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

def gaussian_kernel1d(sigma, length, order=0):
    """
    Computes a 1-D Gaussian convolution kernel.
    """
    if order < 0:
        raise ValueError('order must be non-negative')
    exponent_range = np.arange(order + 1)
    sigma2 = sigma * sigma
    x = np.arange(np.floor(-length/2), np.ceil(length/2))
    phi_x = np.exp(-0.5 / sigma2 * x ** 2)
    phi_x = phi_x / phi_x.sum()
    return phi_x

In [ ]:
probabilities = np.zeros((len(df_test), 30))

normalized_preds = (preds.round()+0.5).astype(int)
for i in np.arange(2, 8, 0.1):
    gaussian_len = 30
    sigma = i
    #sigma = sigma if sigma > 0 else 1
    gaussian = gaussian_kernel1d(sigma, gaussian_len)
    gassian_possibilites = np.zeros((30, 30))
    gaussian_central_point = 15
    for j in range(30):
        target_day = j
        shift_amount = target_day-gaussian_central_point
        probs = shift(gaussian, shift_amount)
        probs = probs/probs.sum()
        gassian_possibilites[j] = probs.round(4)
    
    probabilities = gassian_possibilites[normalized_preds]

    print(sigma, rps(probabilities, ground_truth))

#
#Melhor sigma == 7.1 rps- 4.156727036526029
#

In [ ]:
normalized_preds

In [ ]:
0.5 + 0.4

In [ ]:
probabilities = np.zeros((len(df_test), 30))

#normalized_preds = (preds.round() + 1).astype(int)

gaussian_len = 30
gaussian_central_point = 15
sigma = 7.1

for j, pred in enumerate(preds):
    target_day = pred.round().astype(int)
    target_day = np.clip(target_day, 0, 29)
    #sigma = sigma if sigma > 0 else 1
    #sigma = np.abs(0.5 -(np.abs(target_day - pred))/0.5)*14
    gaussian = gaussian_kernel1d(sigma, gaussian_len)
    shift_amount = target_day-gaussian_central_point
    probs = shift(gaussian, shift_amount)
    probs[target_day+1:] = 0
    probs = probs/probs.sum()
    probabilities[j] = probs.round(4)

print(sigma, rps(probabilities, ground_truth))

In [ ]:
probabilities

In [ ]:
probs

In [ ]:
from scipy.stats import norm

def normal_probs(data):
    row, pred = data
    sku = row['sku']
    target_stock = row['target_stock']
    
    #sold_mean, sold_std = pred
    
    
    days_stockout = pred+1
    std_days = 7

    dist_model = norm(days_stockout,std_days)

    probalities = np.zeros(30)
    for i in range(1, 31):
        probalities[i-1] = (dist_model.cdf(i+1) - dist_model.cdf(i))

    if probalities.sum() == 0:
        probalities = np.ones(30) / 30

    probalities = (probalities/probalities.sum()).round(4)
    #probalities = saferound(probalities, places=4)
    return (sku, probalities)

predictions = np.zeros((len(df_test), 30))
i = 0
with Pool(100) as p:
    for data in tqdm(p.imap(normal_probs, zip(df_test.to_dict(orient='records'), preds)), total=len(df_test)):
        sku, probabilities = data
        predictions[i] = probabilities
        i += 1
        
from evaluate import rps
rps(predictions, ground_truth)

stds
5 - 4.262929225335388
7 - 4.198544959223248
10 - 4.3...
8 - 4.22
6 - 4.204762802167254

In [ ]:
from evaluate import rps
rps(predictions, ground_truth)

In [ ]:
from scipy.stats import norm

def normal_probs(data):
    row, preds = data
    sku = row['sku']
    target_stock = row['target_stock']
    
    sold_mean = preds.mean()
    sold_std = preds.std()
    
    c = preds.cumsum()  
    days_stockout = np.where(c>target_stock)[0]
    days_stockout = 30 if len(days_stockout) == 0 else days_stockout[0]+1
    
    std_days = (sold_std / sold_mean) * days_stockout

    dist_model = norm(days_stockout,std_days)

    probalities = np.zeros(30)
    for i in range(1, 31):
        probalities[i-1] = (dist_model.cdf(i+1) - dist_model.cdf(i))

    if probalities.sum() == 0:
        probalities = np.ones(30) / 30

    probalities = (probalities/probalities.sum()).round(4)
    #probalities = saferound(probalities, places=4)
    return (sku, probalities)

predictions = np.zeros((len(df_test), 30))
i = 0
with Pool(50) as p:
    for data in tqdm(p.imap(normal_probs, zip(df_test.to_dict(orient='records'), y_pred)), total=len(df_test)):
        sku, probabilities = data
        predictions[i] = probabilities
        i += 1